In [3]:
from openai import OpenAI
from pingpongthread import PingPongThread
from keyboard import is_pressed
from time import sleep

client = OpenAI(api_key="api 예시 키")


In [4]:
# PingPong 로봇 연결 및 오디오 장치 확인
PingPong = PingPongThread(number=1, group_id=None)
PingPong.start()

audio_devices = PingPong.get_audio_list()
for i, device in enumerate(audio_devices):
    print(f"{i}: {device}")

# 환경에 맞게 마이크 인덱스 지정
AUDIO_INDEX = 0


Found device: USB 직렬 장치(COM3)
Serial connected.
Write data: DD DD 00 DD 00 00 DA 00 0B 00 00
0: Microsoft Sound Mapper - Input
1: 마이크(Realtek(R) Audio)
2: Microsoft Sound Mapper - Output
3: 스피커(Realtek(R) Audio)
4: 주 사운드 캡처 드라이버
5: 마이크(Realtek(R) Audio)
6: 주 사운드 드라이버
7: 스피커(Realtek(R) Audio)
8: 스피커(Realtek(R) Audio)
9: 마이크(Realtek(R) Audio)
10: 스테레오 믹스 (Realtek HD Audio Stereo input)
11: Speakers 1 (Realtek HDA Primary output with SST)
12: Speakers 2 (Realtek HDA Primary output with SST)
13: PC 스피커 (Realtek HDA Primary output with SST)
14: Headphones 1 (Realtek HD Audio 2nd output with SST)
15: Headphones 2 (Realtek HD Audio 2nd output with SST)
16: PC 스피커 (Realtek HD Audio 2nd output with SST)
17: 마이크 (Realtek HD Audio Mic input)


In [5]:
def ask_medical_chatgpt(user_prompt):
    system_prompt = (
        "당신은 한국의 전문 의료 상담 AI입니다. "
        "사용자의 증상에 대해 신중하고 정확하게 안내하되, "
        "실제 진단이나 처방은 하지 않고 병원 방문을 권유합니다."
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content


In [6]:
PingPong.tts_ko("스페이스 키를 누르면 음성인식을 시작합니다.", is_print=True)

while not is_pressed('esc'):
    if is_pressed(" "):
        print("🎤 마이크를 인식하는 중입니다...")
        result = PingPong.voice_recognize_ko(audio_input_index=AUDIO_INDEX)
        print(f"🧠 사용자 입력: {result}")

        if not result.strip():
            PingPong.tts_ko("말씀을 인식하지 못했어요. 다시 눌러주세요.", is_print=True)
            continue

        # GPT에게 의료 질문 전달
        gpt_response = ask_medical_chatgpt(result)
        print(f"🤖 GPT 응답: {gpt_response}")

        # 로 봇이 GPT 응답 말하기
        PingPong.tts_ko(gpt_response, is_print=True)
        sleep(2)

PingPong.tts_ko("의료 상담을 종료합니다.", is_print=True)


스페이스 키를 누르면 음성인식을 시작합니다.
🎤 마이크를 인식하는 중입니다...
🧠 사용자 입력: 안녕 안녕


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}